In [17]:
# initialise and test speech recognition
import speech_recognition as sr

test_filename = "sample.wav"

r = sr.Recognizer()
data = sr.AudioFile(test_filename)
with data as source:
    audio = r.record(source)

print(r.recognize_google(audio))

I believe you're just talking nonsense


In [18]:
import pyttsx3
tts_engine = pyttsx3.init()

tts_engine.say("checking tts")

tts_engine.runAndWait()

In [19]:
# initialise and test Robot core

from RobotClass import Robot
import time

robot = Robot()

robot.forward(0.3)
time.sleep(0.25)
robot.backward(0.3)
time.sleep(0.25)
robot.stop()

In [22]:
 # Find the correct microphone

device_label = "HD Pro Webcam C920" # for Logitech C920
#device_label = "USB 2.0 Camera: Audio" # for the Nulea cameras
#device_label = "USB Device 0x46d:0x825" # for the logitech c270 camera (non-functional)

try:
    del mic
except NameError:
    pass

for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print("\"{1}\" : {0}".format(index, name))
    if name.startswith(device_label):
        print("Found mic with target name")
        mic = sr.Microphone(device_index=index)
        break
else:
    print("Could not find specified microphone, using default")
    mic = sr.Microphone()       


ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.front.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM surround21
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to

"tegra-hda: HDMI 0 (hw:0,3)" : 0
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,0)" : 1
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,1)" : 2
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,2)" : 3
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,3)" : 4
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,4)" : 5
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,5)" : 6
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,6)" : 7
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,7)" : 8
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,8)" : 9
"tegra-snd-t210ref-mobile-rt565x: - (hw:1,9)" : 10
"HD Pro Webcam C920: USB Audio (hw:2,0)" : 11
Found mic with target name


ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.front.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM surround21
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to

In [29]:
# mk1 code block for voice control

speed_mul = 0.5
turn_mul = 0.5
duration = 1
command_dict = {"forward":[speed_mul,speed_mul,speed_mul,speed_mul],
"backward":[-speed_mul,-speed_mul,-speed_mul,-speed_mul],
"left":[-turn_mul,turn_mul,-turn_mul,turn_mul],
"right":[turn_mul,-turn_mul,turn_mul,-turn_mul]}

similarities = {"backwood":"backward",
"back":"backward",
"light":"right"}

with mic as source:
    #r.adjust_for_ambient_noise(source)
    while True:
        try:
            audio = r.listen(source)

            print("stopped listening, sending audio...")
            recognised_text = r.recognize_google(audio)
            print("speech recognised as:",recognised_text)

            command = recognised_text.split(" ")[0]

            try:
                new_command = similarities[command]
                print("{} sounds pretty similar to {}, substituting.".format(command,new_command))
                command = new_command
            except KeyError:
                pass
            
            print("command:",command)

            try:
                action = command_dict[command]
                tts_engine.say("performing command: {}".format(command))
                tts_engine.runAndWait()

                robot.set_motors(action[0],action[1],action[2],action[3])
                time.sleep(duration)
                robot.stop()

            except KeyError:
                tts_engine.say("I don't recognised the command: {}".format(command))
                tts_engine.runAndWait()
                


        except sr.UnknownValueError:
            pass

stopped listening, sending audio...
speech recognised as: forward
command: forward
stopped listening, sending audio...
speech recognised as: forward forward left
command: forward
stopped listening, sending audio...
stopped listening, sending audio...
speech recognised as: left
command: left
stopped listening, sending audio...
speech recognised as: right
command: right
stopped listening, sending audio...
speech recognised as: right forward
command: right
stopped listening, sending audio...
stopped listening, sending audio...
speech recognised as: forward
command: forward
stopped listening, sending audio...
speech recognised as: backwood
backwood sounds pretty similar to backward, substituting.
command: backward
stopped listening, sending audio...
speech recognised as: left
command: left
stopped listening, sending audio...
speech recognised as: right
command: right
stopped listening, sending audio...
speech recognised as: light
command: light
stopped listening, sending audio...
speech re

KeyboardInterrupt: 